In [1]:
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [42]:
todas_tablas=pd.read_html('https://web.archive.org/web/20210308090120/https://en.wikipedia.org/wiki/2021_Copa_Am%C3%A9rica')

'''cantidad_tablas = len(todas_tablas)
print(f"Cantidad total de tablas: {cantidad_tablas}\n")
tabla_seleccionada = todas_tablas[24]
print(tabla_seleccionada)'''

dict_tablas={}
for letra, i in zip(alfabeto, range(8,25,16)):
    df=todas_tablas[i]
    df.rename(columns={df.columns[1]:'Teams'}, inplace=True)
    df = df[df['Teams'] != 'TBC']
    dict_tablas[f'Group {letra}']=df
dict_tablas['Group B']



,Pos,Teams,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Colombia (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Brazil,0,0,0,0,0,0,0,0,Advance to knockout stage
3,4,Venezuela,0,0,0,0,0,0,0,0,Advance to knockout stage
4,5,Ecuador,0,0,0,0,0,0,0,0,NaN
5,6,Peru,0,0,0,0,0,0,0,0,NaN


In [43]:
with open('dict_table','wb') as output:
    pickle.dump(dict_tablas, output)

In [4]:
def get_matches(tournament, year):
    if tournament == 'SAC':
        web = f'https://en.wikipedia.org/wiki/{year}_South_American_Championship'
    elif tournament == 'CA':
        web = f'https://en.wikipedia.org/wiki/{year}_Copa_Am%C3%A9rica'
    response=requests.get(web)
    content=response.text
    soup=BeautifulSoup(content, 'lxml')
    matches=soup.find_all('div', class_='footballbox')
    
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())
    dict_football={'home': home,'score': score,'away': away}
    df_football=pd.DataFrame(dict_football)
    df_football['year']=year
    return df_football

In [5]:
yearsSAC =[
    1916, 1917, 1919, 1920, 1921, 1922, 1923,
    1924, 1925, 1926, 1927, 1929, 1935, 1937,
    1939, 1941, 1942, 1945, 1946, 1947, 1949,
    1953, 1955, 1956, 1957, 1959, 1959, 1963,
    1967 
]

fifa = [get_matches('SAC', year) for year in yearsSAC]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('copa_america_SAC_historical_data.csv', index=False)

yearsCA = [
    1975, 1979, 1983, 1987, 1989, 1991,
    1993, 1995, 1997, 1999, 2001, 2004, 2007,
    2011, 2015, 2016, 2019
]

fifa = [get_matches('CA', year) for year in yearsCA]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('copa_america_CA_historical_data.csv', index=False)

# Extraer fixture

In [45]:
def get_matches2021(year):
    web='https://web.archive.org/web/20210624225826/https://en.wikipedia.org/wiki/2021_Copa_Am%C3%A9rica'
    response=requests.get(web)
    content=response.text
    soup=BeautifulSoup(content, 'lxml')
    matches=soup.find_all('div', class_='footballbox')
    
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())
    dict_football={'home': home,'score': score,'away': away}
    df_football=pd.DataFrame(dict_football)
    df_football['year']=year
    return df_football
    
df_fixture = get_matches2021('2021')
df_fixture.to_csv('copa_america_fixture.csv', index=False)

# Segunda forma de extraer datos

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

yearsSAC = [
    1916, 1917, 1919, 1920, 1921, 1922, 1923,
    1924, 1925, 1926, 1927, 1929, 1935, 1937,
    1939, 1941, 1942, 1945, 1946, 1947, 1949,
    1953, 1955, 1956, 1957, 1959, 1959, 1963,
    1967
]
'''yearsCA = [
    1975, 1979, 1983, 1987, 1989, 1991,
    1993, 1995, 1997, 1999, 2001, 2004, 2007,
    2011, 2015, 2016, 2019, 2021
]'''
yearsCA = [
    1975, 1979, 1983, 1987, 1989, 1991,
    1993, 1995, 1997, 1999, 2001, 2004, 2007, 
    2016, 2019
]

path = r'chromedriver-win64\chromedriver.exe'
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

def obtener_data_faltante(year, tournament):
    if tournament == 'SAC':
        web = f'https://en.wikipedia.org/wiki/{year}_South_American_Championship'
    elif tournament == 'CA':
        web = f'https://en.wikipedia.org/wiki/{year}_Copa_Am%C3%A9rica'
    driver.get(web)
    home=[]
    score=[]
    away=[]
    
    matches=driver.find_elements(by='xpath',value='//th[@class="fhome"]/.. | //td[@align="right"]/..')
    for match in matches:
        home.append(match.find_element(by='xpath', value='./th[1]').text)
        score.append(match.find_element(by='xpath', value='./th[2]').text)
        away.append(match.find_element(by='xpath', value='./th[3]').text)
    
    dict_football={'home': home, 'score':score, 'away':away}
    df_football=pd.DataFrame(dict_football)
    df_football['year']=year
    time.sleep(2)
    return df_football


In [8]:
fifa=[obtener_data_faltante(year, 'SAC') for year in yearsSAC]
driver.quit()
df_fifa=pd.concat(fifa,ignore_index=True)
df_fifa.to_csv('copa_america_SAC_missing_data.csv',index=False)

In [47]:

'''filtered_years = [year for year in yearsCA if year != 2011 and year != 2015]
fifa=[obtener_data_faltante(year, 'CA') for year in filtered_years]'''
fifa=[obtener_data_faltante(year, 'CA') for year in yearsCA]
driver.quit()
df_fifa=pd.concat(fifa,ignore_index=True)
df_fifa.to_csv('copa_america_CA_missing_data.csv',index=False)